<a href="https://colab.research.google.com/github/PakawanPim/AIT580-HousingProject/blob/main/VerticalSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data pre-processing will happen here. 
Doing an update.

In [1]:
#Installing the required libraries
!pip install geopy
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=e84637a1ccb7428d20b1898147d6f5d32be145d7719b2412da922a7316f78e36
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [3]:
#Importing the required libraries
import pandas as pd
import pandasql as ps
# from pyspark.sql import SparkSession
import os
import sys
import re
from datetime import datetime
from geopy.distance import geodesic as GD

In [6]:
#Upload the data file after pre-processing (only the selected attribues and only above level 240)
from google.colab import files
uploadedfile = files.upload()


Saving airspaceData.csv to airspaceData.csv


In [7]:
import io
airspaceDF = pd.read_csv(io.BytesIO(uploadedfile['airspaceData.csv']))

In [8]:
print(airspaceDF)

                        DateTime  Hour  Minute  Second  microSecond  \
0        2021-12-24 23:59:59.992    23      59      59       992000   
1        2021-12-24 20:54:11.141    20      54      11       141000   
2        2021-12-24 12:36:02.773    12      36       2       773000   
3        2021-12-24 04:13:17.836     4      13      17       836000   
4        2021-12-24 23:59:59.984    23      59      59       984000   
...                          ...   ...     ...     ...          ...   
3303983  2021-12-24 12:36:02.859    12      36       2       859000   
3303984  2021-12-24 12:36:02.852    12      36       2       852000   
3303985  2021-12-24 12:36:02.805    12      36       2       805000   
3303986  2021-12-24 12:36:02.805    12      36       2       805000   
3303987  2021-12-24 12:36:02.789    12      36       2       789000   

          Latitude   Longitude   Height TargetID  SelectedHeading  \
0        21.325208 -158.058116   2200.0   UAL253        78.750000   
1        

In [9]:
#Some column names may be imported with leading spaces. This will remove that
airspaceDF.columns = [x.strip() for x in airspaceDF.columns]

print(airspaceDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3303988 entries, 0 to 3303987
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   DateTime         object 
 1   Hour             int64  
 2   Minute           int64  
 3   Second           int64  
 4   microSecond      int64  
 5   Latitude         float64
 6   Longitude        float64
 7   Height           float64
 8   TargetID         object 
 9   SelectedHeading  float64
 10  TargetAddress    object 
 11  TrackNumber      float64
dtypes: float64(5), int64(4), object(3)
memory usage: 302.5+ MB
None


In [31]:
#create SQL query for flights between the start and end time
sql1 = "SELECT * FROM airspaceDF WHERE Hour = 23 and Minute = 59 ORDER BY TargetID, Second"

#Run query and store results
records2359 = ps.sqldf(sql1, locals())

In [ ]:
#Export the file with just the 23:59 data to make it easier to pick up where we left off
from google.colab import files

records2359.to_csv('records2359_1224.csv', encoding = 'utf-8-sig') 
files.download('records2359_1224.csv')

**START HERE WITH records2359_1224.csv FILE**

In [11]:
#Upload the file with the 23:59 data and save it as a dataframe named records2359

#Upload the records2359_1224.csv data file
from google.colab import files
uploadedfile = files.upload()

import io
records2359 = pd.read_csv(io.BytesIO(uploadedfile['records2359_1224.csv']))

Saving records2359_1224.csv to records2359_1224.csv


In [12]:
#Remove flights without a flight name or altitude
records2359 = records2359.dropna(subset=['TargetID'])
records2359 = records2359.dropna(subset=['Height'])

records2359.head(5)

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
16,16,2021-12-24 23:59:01.875,23,59,1,875000,21.085836,-157.526245,15400.0,AAH24,107.578125,A32F17,1234.0
17,17,2021-12-24 23:59:06.750,23,59,6,750000,21.081926,-157.517598,15400.0,AAH24,107.578125,A32F17,1234.0
18,18,2021-12-24 23:59:10.875,23,59,10,875000,21.078644,-157.510337,15400.0,AAH24,107.578125,A32F17,1234.0
19,19,2021-12-24 23:59:15.102,23,59,15,102000,21.075211,-157.502747,15425.0,AAH24,107.578125,A32F17,1234.0
20,20,2021-12-24 23:59:19.695,23,59,19,695000,21.071594,-157.494807,15400.0,AAH24,107.578125,A32F17,1234.0


In [13]:
#Get the first entry for this minute of time
sql2 = "SELECT *, min(Second) FROM records2359 GROUP BY TargetID"

#Run query and store results
firstEntryDF = ps.sqldf(sql2, locals())
del firstEntryDF['min(Second)']

firstEntryDF.head(5)

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,16,2021-12-24 23:59:01.875,23,59,1,875000,21.085836,-157.526245,15400.0,AAH24,107.578125,A32F17,1234.0
1,30,2021-12-24 23:59:01.508,23,59,1,508000,19.976543,-155.128401,2625.0,AAH504,130.078125,A323F2,1272.0
2,43,2021-12-24 23:59:03.109,23,59,3,109000,25.290344,-147.790578,38775.0,AAL119,232.031250,AB6AFA,1024.0
3,56,2021-12-24 23:59:02.273,23,59,2,273000,42.300705,-132.887941,38200.0,AAL128,99.843750,AB7613,1024.0
4,69,2021-12-24 23:59:01.438,23,59,1,438000,33.262985,-123.632941,30500.0,AAL143,0.000000,A4FBAC,1024.0


In [14]:
#Create tables to store proximity
sql3 = "SELECT * FROM firstEntryDF ORDER BY Latitude"
sql4 = "SELECT * FROM firstEntryDF ORDER BY Longitude"

#Run query and store results
LatitudeOrderDF = ps.sqldf(sql3, locals())
LongitudeOrderDF = ps.sqldf(sql4, locals())


In [15]:
LatitudeOrderDF.head()

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,1100,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,37350.0,FDX77,210.937500,ABE1EB,1024.0
1,1800,2021-12-24 23:59:00.625,23,59,0,625000,6.845460,-163.240356,36000.0,MGE600,NaN,AC3C99,1484.0
2,659,2021-12-24 23:59:03.977,23,59,3,977000,11.171885,178.954757,37225.0,ATN440,51.328125,AA21BF,1024.0
3,1366,2021-12-24 23:59:01.453,23,59,1,453000,11.610489,-166.661035,42300.0,HAL451,NaN,A45233,1024.0
4,2578,2021-12-24 23:59:04.836,23,59,4,836000,12.478409,142.616035,40650.0,SIA8890,258.750000,76B44A,1997.0


In [16]:
LongitudeOrderDF.head()

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,1787,2021-12-24 23:59:03.648,23,59,3,648000,17.969330,-176.990517,38025.0,MGE300,NaN,A9228A,1088.0
1,1100,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,37350.0,FDX77,210.937500,ABE1EB,1024.0
2,3382,2021-12-24 23:59:02.883,23,59,2,883000,33.536041,-175.560315,40950.0,UAL903,288.281250,AA9321,1488.0
3,3368,2021-12-24 23:59:00.430,23,59,0,430000,49.627487,-174.973249,34575.0,UAL893,265.078125,A31A73,1024.0
4,257,2021-12-24 23:59:00.109,23,59,0,109000,47.560684,-172.337723,40800.0,AAL9726,277.031250,AB5257,1024.0


In [17]:
#Calculate the distance of the points closest to each other by longitidue and latitude

longpoint_a = []
longpoint_b = []
longdistance_ab = []

latpoint_a = []
latpoint_b = []
latdistance_ab = []

for a in LongitudeOrderDF.index:   
    for n in range(1,len(LongitudeOrderDF)): 
    # check distance with next flights until found distance > 5, then out of the inner loop
        b = a+n
        if b < len(LongitudeOrderDF):
            point0 = LongitudeOrderDF.loc[a,'Latitude'], LongitudeOrderDF.loc[a,'Longitude']
            pointX = LongitudeOrderDF.loc[b,'Latitude'], LongitudeOrderDF.loc[b,'Longitude']    
            distance = GD(point0,pointX).km  #***change distance calculation***
            if distance < 5: #***change limit closet distance***
                longpoint_a.append(a)
                longpoint_b.append(b)
                longdistance_ab.append(distance)
            else:
                break
print("Points closest to each other by longitiude","\n",longpoint_a,"\n",longpoint_b,"\n", longdistance_ab,"\n")

for a in LatitudeOrderDF.index:   
    for n in range(1,len(LatitudeOrderDF)): 
    # check distance with next flights until found distance > 5, then out of the inner loop
        b = a+n
        if b < len(LatitudeOrderDF):
            point0 = LatitudeOrderDF.loc[a,'Latitude'], LatitudeOrderDF.loc[a,'Longitude']
            pointX = LatitudeOrderDF.loc[b,'Latitude'], LatitudeOrderDF.loc[b,'Longitude']    
            distance = GD(point0,pointX).km  #***change distance calculation***
            if distance < 5: #***change limit closet distance***
                latpoint_a.append(a)
                latpoint_b.append(b)
                latdistance_ab.append(distance)
            else:
                break
print("Points closest to each other by latitiude","\n",latpoint_a,"\n", latpoint_b,"\n", latdistance_ab)

Points closest to each other by longitiude 
 [21, 24, 24, 24, 25, 25, 26, 30, 30, 30, 31, 31, 32, 41, 48, 48, 49, 50, 71, 80, 103, 127, 138, 195] 
 [22, 25, 26, 27, 26, 27, 27, 31, 32, 33, 32, 33, 33, 42, 49, 50, 50, 51, 72, 81, 104, 128, 139, 196] 
 [3.255579188736705, 2.1313517422297488, 3.127667846741136, 4.5643776337660515, 1.434996466892325, 2.471169692567892, 1.7472378177006196, 0.9218110577320656, 3.290574864941219, 2.0669526272726535, 2.3903466132794176, 2.683135906412395, 4.94765010003622, 4.126307979679413, 2.546336772937486, 3.774594340093369, 1.4134566919894227, 4.8861394946873675, 1.4966994126275508, 2.3755551188143795, 3.0801142522943272, 2.842992997732104, 3.188905569758343, 4.048863482140674] 

Points closest to each other by latitiude 
 [10, 27, 49, 50, 58, 80, 88, 90, 90, 90, 91, 91, 92, 132, 143, 207] 
 [11, 28, 50, 51, 59, 81, 89, 91, 92, 93, 92, 93, 93, 133, 144, 208] 
 [3.0801142522943272, 2.3755551188143795, 4.438961860044888, 2.375846821809109, 1.413456691989422

In [18]:
# Create function to select, merge and add the values from analyzing Longitude and Latitude
def newDF(OrderDF,x,y,d):
    """DF is Long/LatitudeOrderDF
       x = long/latpoint_a
       y = long/latpoint_b
       d = long/latdistance_ab"""
    # select rows that index is in list 'point_a', 'point_b'
    A = OrderDF.loc[x,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','Height',
                             'TargetID', 'SelectedHeading', 'TargetAddress']]
    B = OrderDF.loc[y,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','Height',
                             'TargetID', 'SelectedHeading', 'TargetAddress']]
    # Join 2 tables by the "TargetID" of point a (for the uniquness)
    OrderResult = pd.merge(A.reset_index(drop=True),B.reset_index(drop=True),left_index=True, right_index=True)
    # add distance column
    OrderResult['Distance'] = d
    return OrderResult

In [19]:
# Apply function to select and merge data frame
LongOrderResult = newDF(LongitudeOrderDF,longpoint_a, longpoint_b,longdistance_ab)
LatOrderResult = newDF(LatitudeOrderDF,latpoint_a, latpoint_b,latdistance_ab)

In [20]:
# Concatenate results from order by longitude and latitude
Resultsdf = pd.concat([LongOrderResult,LatOrderResult]).reset_index(drop=True)

# Show result
Resultsdf

,DateTime_x,Hour_x,Minute_x,Second_x,microSecond_x,Latitude_x,Longitude_x,Height_x,TargetID_x,SelectedHeading_x,...,Minute_y,Second_y,microSecond_y,Latitude_y,Longitude_y,Height_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Distance
0,2021-12-24 23:59:01.719,23,59,1,719000,22.160156,-159.696317,2450.0,N11QN,NaN,...,59,0,234000,22.160913,-159.664764,1800.0,N11QC,NaN,A02B3B,3.255579
1,2021-12-24 23:59:14.414,23,59,14,414000,22.208908,-159.612821,975.0,N635SB,NaN,...,59,2,523000,22.216095,-159.593645,1175.0,N451H,NaN,A57682,2.131352
2,2021-12-24 23:59:14.414,23,59,14,414000,22.208908,-159.612821,975.0,N635SB,NaN,...,59,2,898000,22.228880,-159.591370,1925.0,N251H,NaN,A25BE4,3.127668
3,2021-12-24 23:59:14.414,23,59,14,414000,22.208908,-159.612821,975.0,N635SB,NaN,...,59,0,16000,22.229438,-159.574433,1725.0,N963SA,NaN,AD68DC,4.564378
4,2021-12-24 23:59:02.523,23,59,2,523000,22.216095,-159.593645,1175.0,N451H,NaN,...,59,2,898000,22.228880,-159.591370,1925.0,N251H,NaN,A25BE4,1.434996
5,2021-12-24 23:59:02.523,23,59,2,523000,22.216095,-159.593645,1175.0,N451H,NaN,...,59,0,16000,22.229438,-159.574433,1725.0,N963SA,NaN,AD68DC,2.471170
6,2021-12-24 23:59:02.898,23,59,2,898000,22.228880,-159.591370,1925.0,N251H,NaN,...,59,0,16000,22.229438,-159.574433,1725.0,N963SA,NaN,AD68DC,1.747238
7,2021-12-24 23:59:01.234,23,59,1,234000,21.955398,-159.347651,750.0,N633JH,NaN,...,59,13,711000,21.963135,-159.344355,1550.0,N140WS,NaN,A0A4B5,0.921811
8,2021-12-24 23:59:01.234,23,59,1,234000,21.955398,-159.347651,750.0,N633JH,NaN,...,59,8,648000,21.984558,-159.341508,150.0,N842JH,NaN,AB8817,3.290575
9,2021-12-24 23:59:01.234,23,59,1,234000,21.955398,-159.347651,750.0,N633JH,NaN,...,59,3,258000,21.940163,-159.336090,625.0,SWA447,355.781250,AB5EB2,2.066953


In [21]:
# Delete duplicate pairs of TargetID x and y
ResultsDF = Resultsdf.drop_duplicates(subset = ['TargetID_x', 'TargetID_y','Distance'],keep = 'last').reset_index(drop = True)
ResultsDF

,DateTime_x,Hour_x,Minute_x,Second_x,microSecond_x,Latitude_x,Longitude_x,Height_x,TargetID_x,SelectedHeading_x,...,Minute_y,Second_y,microSecond_y,Latitude_y,Longitude_y,Height_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Distance
0,2021-12-24 23:59:01.234,23,59,1,234000,21.955398,-159.347651,750.0,N633JH,NaN,...,59,8,648000,21.984558,-159.341508,150.0,N842JH,NaN,AB8817,3.290575
1,2021-12-24 23:59:01.234,23,59,1,234000,21.955398,-159.347651,750.0,N633JH,NaN,...,59,3,258000,21.940163,-159.336090,625.0,SWA447,355.781250,AB5EB2,2.066953
2,2021-12-24 23:59:13.711,23,59,13,711000,21.963135,-159.344355,1550.0,N140WS,NaN,...,59,8,648000,21.984558,-159.341508,150.0,N842JH,NaN,AB8817,2.390347
3,2021-12-24 23:59:13.711,23,59,13,711000,21.963135,-159.344355,1550.0,N140WS,NaN,...,59,3,258000,21.940163,-159.336090,625.0,SWA447,355.781250,AB5EB2,2.683136
4,2021-12-24 23:59:08.648,23,59,8,648000,21.984558,-159.341508,150.0,N842JH,NaN,...,59,3,258000,21.940163,-159.336090,625.0,SWA447,355.781250,AB5EB2,4.947650
5,2021-12-24 23:59:02.023,23,59,2,23000,21.321670,-158.107147,3125.0,UAL253,78.750000,...,59,15,844000,21.308258,-158.070041,75.0,N243SP,NaN,A23CC2,4.126308
6,2021-12-24 23:59:02.586,23,59,2,586000,21.306839,-157.936557,75.0,SWA436,154.687500,...,59,23,148000,21.322042,-157.918142,75.0,STT2040,40.781250,A4A533,2.546337
7,2021-12-24 23:59:02.586,23,59,2,586000,21.306839,-157.936557,75.0,SWA436,154.687500,...,59,2,78000,21.323066,-157.904561,350.0,N506CD,NaN,A65070,3.774594
8,2021-12-24 23:59:02.078,23,59,2,78000,21.323066,-157.904561,350.0,N506CD,NaN,...,59,3,250000,21.293152,-157.869940,425.0,N805NV,NaN,AAF5B0,4.886139
9,2021-12-24 23:59:08.688,23,59,8,688000,21.155777,-156.773155,1825.0,N873SH,NaN,...,59,14,602000,21.169189,-156.771357,2000.0,N752CH,NaN,AA21B1,1.496699


In [24]:
heightDifference = []
potentialLoss = []
potentialLoss1000 = []
potentialLoss400 = []

counter = 0

while counter < len(ResultsDF):
  difference = abs((ResultsDF['Height_x'][counter]) - (ResultsDF['Height_y'][counter]))
  heightDifference.append(difference)

  if difference <= 1000:
    potentialLoss.append('True')
  else:
    potentialLoss.append('False')

  counter = counter + 1

print(heightDifference)
print(potentialLoss)

[600.0, 125.0, 1400.0, 925.0, 475.0, 3050.0, 0.0, 275.0, 75.0, 175.0, 1025.0, 575.0, 5000.0, 575.0, 1025.0, 4150.0, 125.0, 275.0, 800.0, 650.0, 200.0, 950.0, 750.0, 750.0, 550.0, 200.0, 7950.0, 1925.0, 5000.0]
['True', 'True', 'False', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'False', 'True', 'False', 'True', 'False', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'False', 'False']


In [27]:
ResultsDF['HeightDifference'] = heightDifference
ResultsDF['PotentialLoss'] = potentialLoss

print(ResultsDF)

                 DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0   2021-12-24 23:59:01.234      23        59         1         234000   
1   2021-12-24 23:59:01.234      23        59         1         234000   
2   2021-12-24 23:59:13.711      23        59        13         711000   
3   2021-12-24 23:59:13.711      23        59        13         711000   
4   2021-12-24 23:59:08.648      23        59         8         648000   
5   2021-12-24 23:59:02.023      23        59         2          23000   
6   2021-12-24 23:59:02.586      23        59         2         586000   
7   2021-12-24 23:59:02.586      23        59         2         586000   
8   2021-12-24 23:59:02.078      23        59         2          78000   
9   2021-12-24 23:59:08.688      23        59         8         688000   
10  2021-12-24 23:59:07.961      23        59         7         961000   
11  2021-12-24 23:59:01.773      23        59         1         773000   
12  2021-12-24 23:59:02.695      23   

In [30]:
ResultsDF.to_csv('results2359.csv', encoding = 'utf-8-sig') 
files.download('results2359.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM Resultsdf WHERE PotentialLoss = 'True' "

#Run query and store results
LossCandidates = ps.sqldf(sql6, locals())

PandaSQLException: ignored

In [ ]:
print(LossCandidates)

    remove1 DateTime1  Hour1  Minute1  Second1  microSecond1  Latitude1  \
0      1922   59:01.7     23       59        1        719000  22.160156   
1      2262   59:14.4     23       59       14        414000  22.208908   
2      2187   59:02.5     23       59        2        523000  22.216095   
3      2123   59:02.9     23       59        2        898000  22.228880   
4      2249   59:01.2     23       59        1        234000  21.955398   
5      2388   59:08.6     23       59        8        648000  21.984558   
6      2900   59:02.6     23       59        2        586000  21.306839   
7      2596   59:23.1     23       59       23        148000  21.322042   
8      2210   59:02.1     23       59        2         78000  21.323066   
9      2410   59:08.7     23       59        8        688000  21.155777   
10     2223   59:01.8     23       59        1        773000  19.419499   
11     1944   59:04.6     23       59        4        633000  19.396179   
12     1911   59:03.1    